<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/keras/Use_WandbMetricLogger_in_your_Keras_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab-keras-metricslogger} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{intro-colab-keras-metricslogger} -->

# 🏃‍♀️ Introduction
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<img src="http://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<!-- With just a few lines of code,
you'll get rich, interactive, shareable dashboards
like the one in the screenshot below (interactive version
[here](example](https://wandb.ai/stacey/deep-drive?workspace=user-lavanyashukla))).

![](https://i.imgur.com/nHPvS1C.png) -->

This colab notebook introduces the `WandbMetricsLogger` callback. Use this callback for [Experiment Tracking](https://docs.wandb.ai/guides/track). It will log your training and validation metrics along with system metrics to Weights and Biases.



# 🌴 Setup and Installation

First, let us install the latest version of Weights and Biases. We will then authenticate this colab instance to use W&B.

In [1]:
!pip install -qq -U wandb

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds

# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

2023-05-07 16:31:56.376200: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 16:31:56.414887: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 16:31:56.415741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 16:31:57.403159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


If this is your first time using W&B or you are not logged in, the link that appears after running `wandb.login()` will take you to sign-up/login page. Signing up for a [free account](https://wandb.ai/signup) is as easy as a few clicks.

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


True

# 🌳 Hyperparameters

Use of proper config system is a recommended best practice for reproducible machine learning. We can track the hyperparameters for every experiment using W&B. In this colab we will be using simple Python `dict` as our config system.

In [4]:
configs = dict(
    num_classes = 10,
    shuffle_buffer = 1024,
    batch_size = 64,
    image_size = 28,
    image_channels = 1,
    earlystopping_patience = 3,
    learning_rate = 1e-3,
    epochs = 3
)

# 🍁 Dataset

In this colab, we will be using [CIFAR100](https://www.tensorflow.org/datasets/catalog/cifar100) dataset from TensorFlow Dataset catalog. We aim to build a simple image classification pipeline using TensorFlow/Keras.

In [5]:
train_ds, valid_ds = tfds.load('fashion_mnist', split=['train', 'test'])

In [6]:
AUTOTUNE = tf.data.AUTOTUNE


def parse_data(example):
    # Get image
    image = example["image"]
    # image = tf.image.convert_image_dtype(image, dtype=tf.float32)

    # Get label
    label = example["label"]
    label = tf.one_hot(label, depth=configs["num_classes"])

    return image, label


def get_dataloader(ds, configs, dataloader_type="train"):
    dataloader = ds.map(parse_data, num_parallel_calls=AUTOTUNE)

    if dataloader_type=="train":
        dataloader = dataloader.shuffle(configs["shuffle_buffer"])
      
    dataloader = (
        dataloader
        .batch(configs["batch_size"])
        .prefetch(AUTOTUNE)
    )

    return dataloader

In [7]:
trainloader = get_dataloader(train_ds, configs)
validloader = get_dataloader(valid_ds, configs, dataloader_type="valid")

# 🎄 Model

In [8]:
def get_model(configs):
    backbone = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False)
    backbone.trainable = False

    inputs = layers.Input(shape=(configs["image_size"], configs["image_size"], configs["image_channels"]))
    resize = layers.Resizing(32, 32)(inputs)
    neck = layers.Conv2D(3, (3,3), padding="same")(resize)
    preprocess_input = tf.keras.applications.mobilenet.preprocess_input(neck)
    x = backbone(preprocess_input)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(configs["num_classes"], activation="softmax")(x)

    return models.Model(inputs=inputs, outputs=outputs)

In [9]:
tf.keras.backend.clear_session()
model = get_model(configs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 3)         30        
                                                                 
 tf.math.truediv (TFOpLambda  (None, 32, 32, 3)        0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 32, 32, 3)        0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, None, None, 1280)  22579

# 🌿 Compile Model

In [10]:
model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top@5_accuracy')]
)

# 🌻 Train

In [11]:
# Initialize a W&B run
run = wandb.init(
    project = "report-to-notebook",
    config = configs
)

# Train your model
model.fit(
    trainloader,
    epochs = configs["epochs"],
    validation_data = validloader,
    callbacks = [WandbMetricsLogger(log_freq=10)] # Notice the use of WandbMetricsLogger here
)

# Close the W&B run
run.finish()

Epoch 1/3


2023-05-07 16:32:05.659894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-07 16:32:05.660358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


938/938 [==============================] - ETA: 0s - loss: 1.5453 - accuracy: 0.4899 - top@5_accuracy: 0.9047

2023-05-07 16:33:59.803863: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-05-07 16:33:59.804314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


938/938 [==============================] - 121s 126ms/step - loss: 1.5453 - accuracy: 0.4899 - top@5_accuracy: 0.9047 - val_loss: 1.1923 - val_accuracy: 0.6043 - val_top@5_accuracy: 0.9660
Epoch 2/3
938/938 [==============================] - 130s 138ms/step - loss: 1.0961 - accuracy: 0.6292 - top@5_accuracy: 0.9710 - val_loss: 1.0560 - val_accuracy: 0.6378 - val_top@5_accuracy: 0.9738
Epoch 3/3
938/938 [==============================] - 128s 136ms/step - loss: 1.0116 - accuracy: 0.6474 - top@5_accuracy: 0.9744 - val_loss: 1.0076 - val_accuracy: 0.6498 - val_top@5_accuracy: 0.9742


batch/accuracy,▁▂▃▃▄▄▄▄▅▅▅▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▇▆▆▆▅▅▅▅▄▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/top@5_accuracy,▁▂▄▅▅▅▆▆▆▆▆▇▇███████████████████████████
epoch/accuracy,▁▇█
epoch/epoch,▁▅█
epoch/learning_rate,▁▁▁
epoch/loss,█▂▁
epoch/top@5_accuracy,▁██
epoch/val_accuracy,▁▆█


---
panelgrid:
  runsets:
  - project: report-to-notebook
    entity: geekyrakshit
    name: Training-Logs
  lineplots:
  - x: batch/batch_step
    y: batch/accuracy
  - x: batch/batch_step
    y: batch/learning_rate
  - x: batch/batch_step
    y: batch/loss
  - x: batch/batch_step
    y: batch/top@5_accuracy
  - x: epoch/epoch
    y: epoch/accuracy
  - x: epoch/epoch
    y: epoch/learning_rate
  - x: epoch/epoch
    y: epoch/loss
  - x: epoch/epoch
    y: epoch/top@5_accuracy
  - x: epoch/epoch
    y: epoch/val_accuracy
  - x: epoch/epoch
    y: epoch/val_loss
  - x: epoch/epoch
    y: epoch/val_top@5_accuracy
---